In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from subprocess import check_output

In [2]:
medals = pd.read_csv("file:///Users/linhongteng/Downloads/.csv")
medals.info()
medals.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29216 entries, 0 to 29215
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   City          29216 non-null  object
 1   Edition       29216 non-null  int64 
 2   Sport         29216 non-null  object
 3   Discipline    29216 non-null  object
 4   Athlete       29216 non-null  object
 5   NOC           29216 non-null  object
 6   Gender        29216 non-null  object
 7   Event         29216 non-null  object
 8   Event_gender  29216 non-null  object
 9   Medal         29216 non-null  object
dtypes: int64(1), object(9)
memory usage: 2.2+ MB


,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal
0,Athens,1896,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100m freestyle,M,Gold
1,Athens,1896,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100m freestyle,M,Silver
2,Athens,1896,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100m freestyle for sailors,M,Bronze
3,Athens,1896,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100m freestyle for sailors,M,Gold
4,Athens,1896,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100m freestyle for sailors,M,Silver


In [6]:
#1 find the top10 countries ranked by total number of medals
medals_counts = medals["NOC"].value_counts()
print("The total medals: %d" %medals_counts.sum())
print("\nThe top 10 countries:\n", medals_counts.head(10))

The total medals: 29216

The top 10 countries:
 USA    4335
URS    2049
GBR    1594
FRA    1314
ITA    1228
GER    1211
AUS    1075
HUN    1053
SWE    1021
GDR     825
Name: NOC, dtype: int64


In [21]:
#2 find which country won the most medals. (via pivot table)
counted = medals.pivot_table(values="Athlete", index="NOC", columns="Medal", aggfunc="count")
counted["total_medals"] = counted.sum(axis=1)
counted["winning percentage"] = (counted["total_medals"]/medals_counts.sum()).round(2)
counted = counted.sort_values("total_medals", ascending=False)
counted.head(10)

Medal,Bronze,Gold,Silver,total_medals,winning percentage
NOC,,,,,
USA,1052.0,2088.0,1195.0,4335.0,0.15
URS,584.0,838.0,627.0,2049.0,0.07
GBR,505.0,498.0,591.0,1594.0,0.05
FRA,475.0,378.0,461.0,1314.0,0.04
ITA,374.0,460.0,394.0,1228.0,0.04
GER,454.0,407.0,350.0,1211.0,0.04
AUS,413.0,293.0,369.0,1075.0,0.04
HUN,345.0,400.0,308.0,1053.0,0.04
SWE,325.0,347.0,349.0,1021.0,0.03


In [22]:
#3 find which sport is the most competitive. (via pivot table)
counted = medals.pivot_table(index="Sport", values="Athlete", columns="Medal", aggfunc="count")
counted["total_medals"]= counted.sum(axis=1)
counted["issued percentage"] = (counted["total_medals"]/medals_counts.sum()).round(2)
counted = counted.sort_values("total_medals", ascending=False)
counted.head(10)

Medal,Bronze,Gold,Silver,total_medals,issued percentage
Sport,,,,,
Aquatics,1256.0,1302.0,1270.0,3828.0,0.13
Athletics,1137.0,1151.0,1160.0,3448.0,0.12
Rowing,840.0,842.0,841.0,2523.0,0.09
Gymnastics,700.0,789.0,725.0,2214.0,0.08
Fencing,497.0,530.0,520.0,1547.0,0.05
Football,471.0,467.0,449.0,1387.0,0.05
Hockey,427.0,454.0,444.0,1325.0,0.05
Wrestling,392.0,373.0,375.0,1140.0,0.04
Shooting,365.0,369.0,371.0,1105.0,0.04


In [26]:
# search and adjust duplicated columns. (via drop the duplcated to find the unequal rows)
event_gender_uniques = medals[["Gender", "Event_gender"]].drop_duplicates()
event_gender_uniques

,Gender,Event_gender
0,Men,M
348,Men,X
416,Women,W
639,Women,X
23675,Men,W


In [31]:
medals_by_gender = medals.groupby(["Gender", "Event_gender"]).count()
medals_by_gender

City  Edition  Sport  Discipline  Athlete    NOC  Event  \
Gender Event_gender                                                             
Men    M             20067    20067  20067       20067    20067  20067  20067   
       W                 1        1      1           1        1      1      1   
       X              1653     1653   1653        1653     1653   1653   1653   
Women  W              7277     7277   7277        7277     7277   7277   7277   
       X               218      218    218         218      218    218    218   

                     Medal  
Gender Event_gender         
Men    M             20067  
       W                 1  
       X              1653  
Women  W              7277  
       X               218

In [32]:
boolean_filter = (medals.Event_gender == "W") & (medals.Gender == "Men")
medals[boolean_filter]

,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal
23675,Sydney,2000,Athletics,Athletics,"CHEPCHUMBA, Joyce",KEN,Men,marathon,W,Bronze


In [33]:
medals.iloc[[23675]]

,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal
23675,Sydney,2000,Athletics,Athletics,"CHEPCHUMBA, Joyce",KEN,Men,marathon,W,Bronze


In [34]:
medals.iloc[[23675],[6]] = "Women"
medals.iloc[[23675]]

,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal
23675,Sydney,2000,Athletics,Athletics,"CHEPCHUMBA, Joyce",KEN,Women,marathon,W,Bronze


In [35]:
# calculate each country gets medals in how many different sports 
Nsports = medals[["NOC","Sport"]].groupby("NOC", as_index=False).agg({"Sport":"nunique"}).sort_values("Sport", ascending=False)
Nsports.head(10)

,NOC,Sport
130,USA,34
43,GBR,31
41,FRA,28
46,GER,26
21,CHN,24
19,CAN,22
6,AUS,22
35,ESP,22
113,SWE,21
62,ITA,21


In [37]:
# the difference of top 10 countries during cold war (1952~1988)
during_cold_war = (medals.Edition >= 1952) & (medals.Edition <= 1988)
is_top10_countries = medals.NOC.isin(["USA","URS","GBR","FRA","ITA","GER","AUS","HUN","SWE","GDR"])
cold_war_medals = medals.loc[during_cold_war & is_top10_countries]
cold_war_medals

,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal
8018,Helsinki,1952,Aquatics,Diving,"HAASE, Günther",GER,Men,10m platform,M,Bronze
8019,Helsinki,1952,Aquatics,Diving,"LEE, Samuel",USA,Men,10m platform,M,Gold
8021,Helsinki,1952,Aquatics,Diving,"STOVER-IRWIN, Juno Roslays",USA,Women,10m platform,W,Bronze
8022,Helsinki,1952,Aquatics,Diving,"MCCORMICK, Patricia",USA,Women,10m platform,W,Gold
8023,Helsinki,1952,Aquatics,Diving,"MYERS-POPE, Paula Jean",USA,Women,10m platform,W,Silver
...,...,...,...,...,...,...,...,...,...,...
19587,Seoul,1988,Wrestling,Wrestling Gre-R,"TURLYKHANOV, Daulet",URS,Men,68 - 74kg (welterweight),M,Silver
19589,Seoul,1988,Wrestling,Wrestling Gre-R,"MAMIASHVILI, Mikhail",URS,Men,74 - 82kg (middleweight),M,Gold
19590,Seoul,1988,Wrestling,Wrestling Gre-R,"KOMAROMI, Tibor",HUN,Men,74 - 82kg (middleweight),M,Silver
19591,Seoul,1988,Wrestling,Wrestling Gre-R,"POPOV, Vladimir",URS,Men,82 - 90kg (light-heavyweight),M,Bronze


In [38]:
country_grouped = cold_war_medals.groupby("NOC")
list(country_grouped)

[('AUS',
             City  Edition      Sport Discipline  \
  8046   Helsinki     1952   Aquatics   Swimming   
  8123   Helsinki     1952  Athletics  Athletics   
  8124   Helsinki     1952  Athletics  Athletics   
  8136   Helsinki     1952  Athletics  Athletics   
  8193   Helsinki     1952  Athletics  Athletics   
  ...         ...      ...        ...        ...   
  19091     Seoul     1988     Hockey     Hockey   
  19092     Seoul     1988     Hockey     Hockey   
  19093     Seoul     1988     Hockey     Hockey   
  19421     Seoul     1988     Tennis     Tennis   
  19422     Seoul     1988     Tennis     Tennis   
  
                                 Athlete  NOC Gender              Event  \
  8046             DAVIES, John Griffith  AUS    Men  200m breaststroke   
  8123   STRICKLAND-DE LA HUNTY, Shirley  AUS  Women               100m   
  8124                 JACKSON, Marjorie  AUS  Women               100m   
  8136                 JACKSON, Marjorie  AUS  Women            

In [40]:
# each country medals in each yeat. (via pivot table)
medals_won_by_country = medals.pivot_table(index="Edition", values="Athlete", columns="NOC", aggfunc="count", margins=True)
medals_won_by_country

NOC,AFG,AHO,ALG,ANZ,ARG,ARM,AUS,AUT,AZE,BAH,...,URU,USA,UZB,VEN,VIE,YUG,ZAM,ZIM,ZZX,All
Edition,,,,,,,,,,,,,,,,,,,,,
1896,NaN,NaN,NaN,NaN,NaN,NaN,2.0,5.0,NaN,NaN,...,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,6.0,151
1900,NaN,NaN,NaN,NaN,NaN,NaN,5.0,6.0,NaN,NaN,...,NaN,55.0,NaN,NaN,NaN,NaN,NaN,NaN,34.0,512
1904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,394.0,NaN,NaN,NaN,NaN,NaN,NaN,8.0,470
1908,NaN,NaN,NaN,19.0,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,804
1912,NaN,NaN,NaN,10.0,NaN,NaN,NaN,14.0,NaN,NaN,...,NaN,101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,885
1920,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,...,NaN,193.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1298
1924,NaN,NaN,NaN,NaN,11.0,NaN,10.0,4.0,NaN,NaN,...,22.0,198.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,884
1928,NaN,NaN,NaN,NaN,32.0,NaN,4.0,4.0,NaN,NaN,...,22.0,84.0,NaN,NaN,NaN,12.0,NaN,NaN,NaN,710
1932,NaN,NaN,NaN,NaN,4.0,NaN,5.0,5.0,NaN,NaN,...,1.0,181.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,615


In [42]:
# top 10 countries medals during cold war
cold_war_top10_countries_medals = medals_won_by_country.loc[1952:1988, ["USA","URS","GBR","FRA","ITA","GER","AUS","HUN","SWE","GDR"]]
cold_war_top10_countries_medals

NOC,USA,URS,GBR,FRA,ITA,GER,AUS,HUN,SWE,GDR
Edition,,,,,,,,,,
1952,130.0,117.0,31.0,39.0,53.0,40.0,20.0,101.0,70.0,NaN
1956,118.0,169.0,45.0,32.0,46.0,NaN,61.0,64.0,34.0,NaN
1960,112.0,169.0,28.0,11.0,88.0,NaN,36.0,65.0,7.0,NaN
1964,150.0,174.0,28.0,31.0,50.0,NaN,42.0,56.0,15.0,NaN
1968,149.0,188.0,18.0,27.0,32.0,NaN,49.0,81.0,9.0,51.0
1972,155.0,211.0,29.0,22.0,25.0,NaN,20.0,81.0,21.0,148.0
1976,155.0,285.0,32.0,20.0,31.0,NaN,24.0,55.0,10.0,192.0
1980,NaN,442.0,45.0,29.0,37.0,NaN,12.0,61.0,18.0,260.0
1984,333.0,NaN,72.0,68.0,63.0,NaN,50.0,NaN,32.0,NaN


In [58]:
print('Consistency during cold war\n',cold_war_top10_countries_medals.idxmax(axis=1))
print('\nTotal counts\n', cold_war_top10_countries_medals.idxmax(axis=1).value_counts())

Consistency during cold war
 Edition
1952    USA
1956    URS
1960    URS
1964    URS
1968    URS
1972    URS
1976    URS
1980    URS
1984    USA
1988    URS
dtype: object

Total counts
 URS    8
USA    2
dtype: int64


In [61]:
# top 10 countries medals after cold war
after_cold_war_top10_countries_medals = medals_won_by_country.loc[1992:2008, ["USA","URS","GBR","FRA","ITA","GER","AUS","HUN","SWE","GDR"]]
after_cold_war_top10_countries_medals

NOC,USA,URS,GBR,FRA,ITA,GER,AUS,HUN,SWE,GDR
Edition,,,,,,,,,,
1992,224.0,NaN,50.0,57.0,46.0,198.0,57.0,45.0,35.0,NaN
1996,260.0,NaN,26.0,51.0,71.0,124.0,132.0,43.0,31.0,NaN
2000,248.0,NaN,55.0,66.0,65.0,119.0,183.0,53.0,32.0,NaN
2004,264.0,NaN,57.0,53.0,102.0,149.0,157.0,40.0,12.0,NaN
2008,315.0,NaN,77.0,76.0,42.0,101.0,149.0,27.0,7.0,NaN


In [65]:
after_cold_war_top10_countries_medals.pct_change().round(2)


NOC,USA,URS,GBR,FRA,ITA,GER,AUS,HUN,SWE,GDR
Edition,,,,,,,,,,
1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,0.16,NaN,-0.48,-0.11,0.54,-0.37,1.32,-0.04,-0.11,NaN
2000,-0.05,NaN,1.12,0.29,-0.08,-0.04,0.39,0.23,0.03,NaN
2004,0.06,NaN,0.04,-0.20,0.57,0.25,-0.14,-0.25,-0.62,NaN
2008,0.19,NaN,0.35,0.43,-0.59,-0.32,-0.05,-0.32,-0.42,NaN


In [66]:
print('Consistency after cold war\n',after_cold_war_top10_countries_medals.idxmax(axis=1))
print('\nTotal counts\n', after_cold_war_top10_countries_medals.idxmax(axis=1).value_counts())

Consistency after cold war
 Edition
1992    USA
1996    USA
2000    USA
2004    USA
2008    USA
dtype: object

Total counts
 USA    5
dtype: int64
